In [55]:
import pandas as pd
import numpy as np

In [56]:
pretrain_mut = pd.read_csv('../data/pdx/pdx_mut.csv', index_col=0)
pretrain_mut.head()

,A2LD1,ABCA4,ABCC13,ABHD13,ABTB1,ABTB2,ACBD7,ACTR6,ADAM17,ADAM3A,...,FAM196A,GPR123,MIR202,NKX6-2,PCDHGC5,SEPT2,SEPT6,SEPT8,UTF1,VENTX
X-1004,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
X-1008,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1298,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
X-1349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# data = pd.read_csv('../data/pdx/PDX_klaegerscan_data_for_ml.csv')
# print(data.shape)
# data.to_feather('../data/pdx/PDX_klaegerscan_data_for_ml.feather')

(1415, 33204)


,Model,Treatment,Treatment.target,Treatment.type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,...,mut_ZDHHC11_0,mut_ZDHHC11_1,mut_ZNF219_0,mut_ZNF219_1,mut_ZNF384_0,mut_ZNF384_1,mut_LOC728888_1,mut_LOC728888_0,mut_NBPF12_0,mut_NBPF12_1
0,X1004,BGJ398,FGFR,single,-54.3,12,-28.1,26,26.0,26,...,1,0,1,0,1,0,1,0,1,0
1,X1004,BKM120,"PIK3CA,PIK3CB,PIK3CG,PIK3CD,panPI3K",single,64.6,22,23.7,11,14.9,27,...,1,0,1,0,1,0,1,0,1,0
2,X1004,BYL719,PIK3CA,single,20.0,18,9.3,11,32.3,42,...,1,0,1,0,1,0,1,0,1,0
3,X1004,BYL719,PIK3CA,single,20.0,18,9.3,11,32.3,42,...,1,0,1,0,1,0,1,0,1,0
4,X1004,BYL719 + LEE011,"PIK3CA,CDK4/6",combo,119.0,22,52.6,11,16.2,35,...,1,0,1,0,1,0,1,0,1,0


In [ ]:
data = pd.read_feather('../data/pdx/PDX_klaegerscan_data_for_ml.feather')

In [59]:
#get columns that do not start with 'mut_' or 'act_' or 'cna_' or 'exp_'
mut_col = [col for col in data.columns if 'mut_' in col]
act_col = [col for col in data.columns if 'act_' in col]
cna_col = [col for col in data.columns if 'cnv_' in col]
exp_col = [col for col in data.columns if 'exp_' in col]
id_cols = []
for col in data.columns:
    if col not in mut_col and col not in act_col and col not in cna_col and col not in exp_col:
        id_cols.append(col)
len(id_cols)


17

In [60]:
data_id = data[id_cols]
data_id.head()

,Model,Treatment,Treatment.target,Treatment.type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,ResponseCategory,NonPD,CRorPR,TumorType,RespScaled,logSurv,logSurvScaled
0,X1004,BGJ398,FGFR,single,-54.3,12,-28.1,26,26.0,26,PR,1,1,BRCA,-0.505263,3.258097,3.369223
1,X1004,BKM120,"PIK3CA,PIK3CB,PIK3CG,PIK3CD,panPI3K",single,64.6,22,23.7,11,14.9,27,PD,0,0,BRCA,0.426147,2.701361,2.793499
2,X1004,BYL719,PIK3CA,single,20.0,18,9.3,11,32.3,42,SD-->PD,1,0,BRCA,0.167222,3.475067,3.593594
3,X1004,BYL719,PIK3CA,single,20.0,18,9.3,11,32.3,42,SD-->PD,1,0,BRCA,0.167222,3.475067,3.593594
4,X1004,BYL719 + LEE011,"PIK3CA,CDK4/6",combo,119.0,22,52.6,11,16.2,35,PD,0,0,BRCA,0.945795,2.785011,2.880002


In [61]:
data_act = data[act_col]
data_act['CRorPR'] = data['CRorPR']
data_act['CRorPR'].value_counts()


/tmp/ipykernel_2851171/2884522922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['CRorPR'] = data['CRorPR']


CRorPR
0    1261
1     154
Name: count, dtype: int64

In [62]:
#reverse the labels for 'CRorPR', 0 -> 1, 1 -> 0
data_act['CRorPR'] = data_act['CRorPR'].apply(lambda x: 1 if x == 0 else 0)
data_act['CRorPR'].value_counts()

/tmp/ipykernel_2851171/1627273038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['CRorPR'] = data_act['CRorPR'].apply(lambda x: 1 if x == 0 else 0)


CRorPR
1    1261
0     154
Name: count, dtype: int64

In [63]:
data_act.shape

(1415, 818)

In [64]:
clin = pd.read_csv('../data/crc_ib_mut_cna_fus_act_clin.csv', index_col=0)
clin.head()

,id_institution,id_drugs_list,OS,PFS,id_tt_os_g_mos,id_tt_pfs_m_g_mos,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,...,act_PTK2B,act_RPS6KB1,act_STK10,act_STK11,act_STK24,act_STK3,act_STK38,act_STK38L,act_STK4,act_YES1
GENIE-DFCI-000971,DFCI,Regorafenib,1,1.0,0.625000,0.361842,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-001038,DFCI,Regorafenib,0,1.0,23.223684,3.157895,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002499,DFCI,Cabozantinib Smalate,1,0.0,8.881579,0.921053,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002507,DFCI,Regorafenib,1,1.0,4.802632,2.697368,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002561,DFCI,Regorafenib,1,1.0,10.789474,4.605263,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [65]:
mut_clin = [col for col in clin.columns if 'mut_' in col]
mut_clin_genes = [col.split('_')[1] for col in mut_clin]
mut_data = [col for col in pretrain_mut.columns]
overlap = [col for col in mut_clin_genes if col in mut_data]
len(overlap)

219

In [66]:
pretrain_mut = pretrain_mut[overlap]
mut_clin = ['mut_' + col for col in overlap]
clin_mut = clin[mut_clin]
print(pretrain_mut.shape, clin_mut.shape)

(399, 219) (139, 219)


In [67]:
for col in mut_clin:
    data_act[col] = np.NaN

/tmp/ipykernel_2851171/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act[col] = np.NaN
/tmp/ipykernel_2851171/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act[col] = np.NaN
/tmp/ipykernel_2851171/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [68]:
pretrain_mut.index = pretrain_mut.index.str.replace('-', '')
pretrain_mut.head()

,CDK4,CCND3,CDH1,CDK8,GNAS,PRKAR1A,MLH1,PNRC1,ETV1,TSC1,...,CDKN1A,ROS1,DAXX,TSC2,RAD51C,ARID1A,AURKB,U2AF1,REL,BRCA1
X1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1298,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
X1349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
data_act.head()

,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_CDKN1A,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1
0,0.7600,1.0,0.5500,0.640,0.62000,0.90,0.89,1.00,1.0000,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0000,1.0,0.6800,1.000,1.00000,1.00,0.82,0.75,0.7200,0.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0000,1.0,1.0000,1.000,1.00000,1.00,1.00,0.93,1.0000,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.7500,1.0,0.5300,0.100,0.96000,1.00,0.93,1.00,0.9300,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.5925,1.0,0.4399,0.085,0.05568,0.96,0.93,0.90,0.8928,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
data_act['line'] = data['Model']
data_act.head()

/tmp/ipykernel_2851171/3177637675.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_act['line'] = data['Model']
/tmp/ipykernel_2851171/3177637675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['line'] = data['Model']


,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1,line
0,0.7600,1.0,0.5500,0.640,0.62000,0.90,0.89,1.00,1.0000,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1004
1,1.0000,1.0,0.6800,1.000,1.00000,1.00,0.82,0.75,0.7200,0.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1004
2,1.0000,1.0,1.0000,1.000,1.00000,1.00,1.00,0.93,1.0000,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1004
3,0.7500,1.0,0.5300,0.100,0.96000,1.00,0.93,1.00,0.9300,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1004
4,0.5925,1.0,0.4399,0.085,0.05568,0.96,0.93,0.90,0.8928,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1004


In [71]:
#fill in the 'mut_' columns of data_act with the pretrain_mut, matching the index of pretrain_mut with 'line' in data_act
for i in range(data_act.shape[0]):
    line = data_act.iloc[i]['line']
    if line in pretrain_mut.index:
        data_act.loc[i, mut_clin] = pretrain_mut.loc[line].values

In [72]:
data_act.head()

,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1,line
0,0.7600,1.0,0.5500,0.640,0.62000,0.90,0.89,1.00,1.0000,0.74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,X1004
1,1.0000,1.0,0.6800,1.000,1.00000,1.00,0.82,0.75,0.7200,0.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,X1004
2,1.0000,1.0,1.0000,1.000,1.00000,1.00,1.00,0.93,1.0000,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,X1004
3,0.7500,1.0,0.5300,0.100,0.96000,1.00,0.93,1.00,0.9300,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,X1004
4,0.5925,1.0,0.4399,0.085,0.05568,0.96,0.93,0.90,0.8928,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,X1004


In [73]:
data_act.shape

(1415, 1038)

In [74]:
data_act.set_index('line', inplace=True, drop=True)


In [75]:
data_act.reset_index(inplace=True)
data_act.to_csv('../data/pdx/pdx_act_mut_cpr_total.csv')

/tmp/ipykernel_2851171/2207424860.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_act.reset_index(inplace=True)


In [46]:
data = pd.read_csv('../data/pdx/PDX_klaegerscan_data_for_ml_CRC.csv')
print(data.shape)
#get columns that do not start with 'mut_' or 'act_' or 'cna_' or 'exp_'
mut_col = [col for col in data.columns if 'mut_' in col]
act_col = [col for col in data.columns if 'act_' in col]
cna_col = [col for col in data.columns if 'cnv_' in col]
exp_col = [col for col in data.columns if 'exp_' in col]
id_cols = []
for col in data.columns:
    if col not in mut_col and col not in act_col and col not in cna_col and col not in exp_col:
        id_cols.append(col)
len(id_cols)

data_id = data[id_cols]
data_id.head()
data_act = data[act_col]
data_act['CRorPR'] = data['CRorPR']
data_act['CRorPR'].value_counts()

#reverse the labels for 'CRorPR', 0 -> 1, 1 -> 0
data_act['CRorPR'] = data_act['CRorPR'].apply(lambda x: 1 if x == 0 else 0)
data_act['CRorPR'].value_counts()
data_act.shape
clin = pd.read_csv('../data/crc_ib_mut_cna_fus_act_clin.csv', index_col=0)
clin.head()
mut_clin = [col for col in clin.columns if 'mut_' in col]
mut_clin_genes = [col.split('_')[1] for col in mut_clin]
mut_data = [col for col in pretrain_mut.columns]
overlap = [col for col in mut_clin_genes if col in mut_data]
len(overlap)
pretrain_mut = pretrain_mut[overlap]
mut_clin = ['mut_' + col for col in overlap]
clin_mut = clin[mut_clin]
print(pretrain_mut.shape, clin_mut.shape)
for col in mut_clin:
    data_act[col] = np.NaN
pretrain_mut.index = pretrain_mut.index.str.replace('-', '')
pretrain_mut.head()
data_act.head()
data_act['line'] = data['Model']
data_act.head()
#fill in the 'mut_' columns of data_act with the pretrain_mut, matching the index of pretrain_mut with 'line' in data_act
for i in range(data_act.shape[0]):
    line = data_act.iloc[i]['line']
    if line in pretrain_mut.index:
        data_act.loc[i, mut_clin] = pretrain_mut.loc[line].values
data_act.head()
data_act.shape
data_act.set_index('line', inplace=True)
data_act.reset_index(inplace=True, drop=True)
data_act.to_csv('../data/pdx/pdx_act_mut_cpr_crc.csv')

(329, 33204)


/tmp/ipykernel_2851171/2355467840.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['CRorPR'] = data['CRorPR']
/tmp/ipykernel_2851171/2355467840.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['CRorPR'] = data_act['CRorPR'].apply(lambda x: 1 if x == 0 else 0)
/tmp/ipykernel_2851171/2355467840.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

(399, 219) (139, 219)


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [48]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
aurocs = cross_val_score(rf, data_act.drop(['CRorPR'], axis=1), data_act['CRorPR'], scoring='roc_auc', cv=5)
aurocs

array([0.71370968, 0.66393443, 0.54590164, 0.48852459, 0.90983607])

In [49]:
np.mean(aurocs)

0.664381279746166

In [50]:
act_cols = [col for col in data_act.columns if 'act_' in col]
#combine the 'act_' columns with the 'mut_' columns
act_mut_cols = act_cols + mut_clin
len(act_mut_cols)

1036

In [51]:
train = data_act.copy()
test = clin[act_mut_cols]
test['OS'] = clin['OS']
test['PFS'] = clin['PFS']
test.to_csv('../data/crc_ib_mut_act.csv')

/tmp/ipykernel_2851171/1423953895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['OS'] = clin['OS']
/tmp/ipykernel_2851171/1423953895.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['PFS'] = clin['PFS']


In [54]:
X_train = train.drop(['CRorPR'], axis=1)
y_train = train['CRorPR']
X_test = test.drop(['OS', 'PFS'], axis=1)
y_test = test['OS']

In [53]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.47773405136702574

In [44]:
train = data_act.copy()
test = clin[act_mut_cols]
test['PFS'] = clin['PFS']
test = test.dropna(subset=['PFS'])
print(train.shape, test.shape)

(1415, 1038) (133, 1037)


/tmp/ipykernel_2851171/3769482909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['PFS'] = clin['PFS']


In [ ]:
X_train = train.drop(['line', 'CRorPR'], axis=1)
y_train = train['CRorPR']
X_test = test.drop(['PFS', 'OS'], axis=1)
y_test = test['PFS']
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.4570990806945863

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
aurocs = cross_val_score(rf, X_test, y_test, scoring='roc_auc', cv=5)
aurocs

array([0.45061728, 0.57407407, 0.62962963, 0.62152778, 0.60130719])

In [ ]:
np.mean(aurocs)

0.5754311909949166